# Using rabbitmq-server Message Broker with Amazon AWS

### Installation

### check notes on http://kronosapiens.github.io/blog/2015/04/28/rabbitmq-aws.html

Install rabbitmq-server server first:

In [1]:
#!sudo apt-get install rabbitmq-server

check if the rabbitmq server is up and runnning: 

In [2]:
!sudo rabbitmqctl status

Status of node 'rabbit@ip-172-31-2-17' ...
[{pid,1968},
 {running_applications,[{rabbit,"RabbitMQ","3.5.7"},
                        {mnesia,"MNESIA  CXC 138 12","4.13.3"},
                        {xmerl,"XML parser","1.3.10"},
                        {os_mon,"CPO  CXC 138 46","2.4"},
                        {sasl,"SASL  CXC 138 11","2.7"},
                        {stdlib,"ERTS  CXC 138 10","2.8"},
                        {kernel,"ERTS  CXC 138 10","4.2"}]},
 {os,{unix,linux}},
 {erlang_version,"Erlang/OTP 18 [erts-7.3] [source] [64-bit] [smp:4:4] [async-threads:64] [kernel-poll:true]\n"},
 {memory,[{total,59036720},
          {connection_readers,9072},
          {connection_writers,2784},
          {connection_channels,6016},
          {connection_other,35032},
          {queue_procs,31592},
          {queue_slave_procs,0},
          {plugins,0},
          {other_proc,13622088},
          {mnesia,68136},
          {mgmt_db,0},
          {msg_index,294784},
          {other_ets,858336}

RabbitMQ comes with a “guest” user out of the box, but this user is not configured to accept external requests. This would be fine if you were planning on sending and consuming messages locally, but we are not.

The easiest way to configure RabbitMQ to accept external requests is to create a new user. Go ahead and run the following. (Feel free to chahnge the myguest username and password to anytihng you like):

In [3]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

This will create a new user on the RabbitMQ server. We will use this username and password to configure the app-layer instance to send messages to Rabbit.

Let’s check to make sure the user was created correctly:

In [4]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


Pika is a pure-Python implementation of the AMQP 0-9-1 protocol that is used by RabbitMQ.

### For more details see  https://www.rabbitmq.com/tutorials/tutorial-one-python.html

In [5]:
#!pip install pika --upgrade

In [6]:
import pika

In [7]:
pika.__version__

'1.1.0'

### Example for sending message through web

Below I'm using a balance loader (PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com) that I set up on AWS. <br>
<b> You should change it to your machine's address!

Before you run the next cell, make sure AWS "Target groups" in the "Load balancing" are set up correctly and the instances are registered. 

In [8]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
parameters =  pika.ConnectionParameters('PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com', port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

<b> Before you run the next cell, you should also run the receiver: "receive_through_url.ipynb"

In [9]:
channel.basic_publish(exchange='',
                      routing_key='hello2',
                      body='Hello World3!')
print(" [x] Sent 'Hello World3!'")

 [x] Sent 'Hello World3!'


In [10]:
connection.close()